<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#1.-Setting-Parameter" data-toc-modified-id="1.-Setting-Parameter-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. Setting Parameter</a></span></li><li><span><a href="#2.-Data-Collect" data-toc-modified-id="2.-Data-Collect-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. Data Collect</a></span></li><li><span><a href="#3.-Example-Input-Data" data-toc-modified-id="3.-Example-Input-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3. Example Input Data</a></span></li><li><span><a href="#4.-Example-Save-Data" data-toc-modified-id="4.-Example-Save-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>4. Example Save Data</a></span></li></ul></div>

##### CSV 타입의 Data에서 특정 조건에 만족하는 데이터만을 업로드하는 Test Code

----------
- CSV 형태의 데이터파일 업로드 Test Code
- CSV Data Upload 방식에는 4가지 타입이 존재
    1. Basic : 원 데이터를 변형 없이 그대로 업로드하거나 컬럼을 삭제 및 이름 변경 후 업로드하는 방법
    2. **Selected Data : 원 데이터에서 특정 컬럼이 특정 조건을 만족하는 데이터만을 선택하여 업로드하는 방식 (1번 타입의 방법 기능을 포함)**
        - 특정 조건 종류
            - Equal/Above/Below/Exceeded/Less than/Exception
    3. TimeDuplication Data : 시간정보가 중복되어 정보가 수집된 데이터로 중복된 시간을 처리하여 업로드하는 방법 (1,2번 타입의 방법 기능을 포함)
        - 중복 처리 방법
            -Remove/Sum/Average/Max/Min
    4. Folder Data (highCapacity) : 하나의 폴더안에 여러개의 데이터가 저장되어 있으며 폴더로 데이터를 업로드하는 방법 (대용량 업로드/1,2번 타입의 기능을 포함)

# Import

In [1]:
import pandas as pd
import os
import sys

sys.path.append("../")
sys.path.append("../..")

from Clust.setting.influx_setting_KETI import CLUSTDataServer2 as ins
from Clust.clust.ingestion.influx import influx_Client_v2 as iC

import Clust.clust.ingestion.CSVtoInflux.csvCollector as cC

# 1. Setting Parameter

- parameter 는 KETIPreDataCollection.collectorParameter 의 parameter 로 호출 가능
```
###예시###
import KETIPreDataCollection.collectorParameter as cP
cP.parameter
```

In [2]:
####### db connection parameter
db_client = iC.influxClient(ins)

In [3]:
####### Folder / File 업로드 방식 고르시
upload_type_list = ["File", "Folder"]
upload_type = upload_type_list[0]

In [4]:
####### read&clean type 고르기 parameter
data_read_type_list = ["basic", "selectedData", "highCapacity"]
data_read_type = data_read_type_list[1]

In [5]:
####### 기본 Data 관련 저장에 필요한 정보 기입
db_name = "test_data_type2"
ms_name = "test2"
path = "./exampleData/Type2_SelectedData.csv"
encoding = "utf-8"

In [6]:
####### 저장될 데이터의 시간 스템프로 지정될 column 을 기입하는 parameter
time_column = "date"

In [7]:
####### clean type 에 따라 data read type의 parameter 가 달라짐
selected_datas = [
    {"Selected_columns":["nation"]},
    {"Selected_values":["Taiwan"]},
    {"Selected_Function":["Equal"]}
]

In [8]:
####### 저장하는 column 선택하는 parameter
selected_columns = ['visitor', 'age0-20', 'age21-30', 'age31-40', 'age41-50']

In [9]:
####### duplicated time column processing method 정하는 parameter
dtcpm = None

In [10]:
####### 저장 데이터의 field type 지정하는 parameter
field_type = "float"

In [11]:
####### 최종 Parameter
parameter = {
    "uploadType" : upload_type,
    "dataReadType" : data_read_type,
    "dbName" : db_name,
    "msName" : ms_name,
    "filePath" : path,
    "selectedDatas" : selected_datas, # data read type 이 selectedData 인 경우 필요한 param
    "timeColumnName" : time_column,
    "selectedColumns" : selected_columns,
    "duplicatedTimeColumnProcessingMethod" : dtcpm,
    "encoding" : encoding,
    "fieldType" : field_type
}
print("parameter success")

parameter success


# 2. Data Collect

In [12]:
csv_collector = cC.Collector(parameter, db_client)
csv_collector.collect()

===========data clean success===========
Writing Data ...
=========== ms name : test2 ===========
            visitor  age0-20  age21-30  age31-40  age41-50
time                                                      
2020-03-01    585.0     14.0     232.0     128.0      60.0
2020-04-01    155.0      6.0      38.0      33.0      14.0
========== create bucket ==========
========== write success ==========


# 3. Example Input Data

In [13]:
ex_data = pd.read_csv(path)

In [14]:
ex_data.head()

,date,nation,visitor,growth,share,age0-20,age21-30,age31-40,age41-50,age51-60,age61
0,2019-1,China,392814,28.737870,35.555117,36520,108591,103657,48574,40893,40998
1,2019-1,Japan,206526,23.606830,18.693468,18015,57921,34165,39811,33857,20330
2,2019-1,Taiwan,87954,16.003693,7.961057,18888,17927,18595,18862,8169,4566
3,2019-1,Hong Kong,35896,3.533212,3.249086,3890,11384,7400,5461,4629,2137
4,2019-1,Macao,2570,-12.376406,0.232621,223,1013,762,264,181,92


# 4. Example Save Data 

In [15]:
db_client = iC.influxClient(ins)
save_data = db_client.get_data(db_name, ms_name)

In [16]:
save_data.head()

,age0-20,age21-30,age31-40,age41-50,visitor
time,,,,,
2019-01-01 00:00:00+00:00,18888.0,17927.0,18595.0,18862.0,87954.0
2019-02-01 00:00:00+00:00,15862.0,22177.0,20705.0,17971.0,94559.0
2019-03-01 00:00:00+00:00,7613.0,24934.0,25101.0,17349.0,98510.0
2019-04-01 00:00:00+00:00,9518.0,23691.0,28047.0,20239.0,113072.0
2019-05-01 00:00:00+00:00,10262.0,21375.0,25044.0,18819.0,101779.0
